# 02 - Model Training and Walk-Forward

This notebook shows how to:

- Build training tensors from OHLCV data
- Train LSTM / TCN / Transformer models
- Run a simple walk-forward backtest
- Inspect basic performance metrics



In [1]:
import sys
from pathlib import Path

ROOT = Path("..").resolve()
print("ROOT:", ROOT, "exists?", ROOT.exists())
print("ROOT/research exists?", (ROOT / "research").exists())

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

ROOT: /home/giorgos/Desktop/AlphaFactory exists? True
ROOT/research exists? True


In [ ]:
import pathlib
import numpy as np
import pandas as pd
from research.common import load_ohlcv_csv, BarConfig
from research.feature_engineering import FeatureConfig, build_features
from research.model_lstm import LSTMAlpha, LSTMConfig, train_lstm
from research.model_tcn import TCNAlpha, TCNConfig, train_tcn
from research.model_transformer import TransformerAlpha, TransformerConfig, train_transformer
from research.walk_forward import WalkForwardConfig, walk_forward
from research.evaluation import aggregate_walk_forward

DATA_PATH = pathlib.Path("../data/equities/AAPL.csv")

raw = load_ohlcv_csv(str(DATA_PATH))
print(raw.dtypes)
print(raw.head())
num_cols = ["open", "high", "low", "close", "volume"]

# Strip whitespace and force numeric for each, turning bad values into NaN
for c in num_cols:
    raw[c] = (
        raw[c]
        .astype(str)
        .str.replace(",", "", regex=False)  # drop thousands separators if any
        .str.strip()
    )
    raw[c] = pd.to_numeric(raw[c], errors="coerce")

# Drop any rows where OHLCV is missing
raw = raw.dropna(subset=num_cols).reset_index(drop=True)

print(raw[num_cols].dtypes)
print(raw[num_cols].head())

bar_cfg = BarConfig(lookback_window=128, prediction_horizon=5)
feat_cfg = FeatureConfig(bar=bar_cfg)
X, y, feature_cols = build_features(raw, feat_cfg)

timestamp    datetime64[ns]
open                 object
high                 object
low                  object
close                object
volume               object
dtype: object
   timestamp                open                high                 low  \
0 2015-01-02   24.69423503534699   24.70532029541843   23.79860048379208   
1 2015-01-05  24.006990190147395    24.0867993234754  23.368518814896508   
2 2015-01-06   23.61903255400942  23.816338020280458   23.19560061965636   
3 2015-01-07   23.76535220011818   23.98704392471396   23.65450633782029   
4 2015-01-08   24.21537773680298  24.862716897095112  24.097879750947854   

                close     volume  
0  24.237550735473633  212818400  
1  23.554738998413086  257142000  
2   23.55695915222168  263188400  
3  23.887283325195312  160423600  
4  24.805076599121094  237458000  
open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object
        open       high        low      close  

In [3]:
# Choose a model family here: "lstm", "tcn", or "transformer"
MODEL_FAMILY = "lstm"

n_features = X.shape[-1]
len(X)


def train_fn(train_data, val_data):
    X_train, y_train = train_data
    X_val, y_val = val_data

    if MODEL_FAMILY == "lstm":
        cfg = LSTMConfig(input_size=n_features)
        model = LSTMAlpha(cfg)
        return train_lstm(model, (X_train, y_train), (X_val, y_val), epochs=5)
    elif MODEL_FAMILY == "tcn":
        cfg = TCNConfig(input_size=n_features)
        model = TCNAlpha(cfg)
        return train_tcn(model, (X_train, y_train), (X_val, y_val), epochs=5)
    elif MODEL_FAMILY == "transformer":
        cfg = TransformerConfig(input_size=n_features)
        model = TransformerAlpha(cfg)
        return train_transformer(model, (X_train, y_train), (X_val, y_val), epochs=5)
    else:
        raise ValueError(f"Unknown MODEL_FAMILY={MODEL_FAMILY}")



In [6]:
from research.walk_forward import WalkForwardConfig, walk_forward

wf_cfg = WalkForwardConfig(window_train=500, window_val=100, step=100)  # smaller windows if data is short
wf_results = walk_forward(X, y, wf_cfg, train_fn)

print(type(wf_results), len(wf_results))  # should be <class 'list'> and > 0

<class 'list'> 19


In [7]:
summary = aggregate_walk_forward(wf_results)
stats = summary["stats"]
stats

PerformanceStats(sharpe=-0.4755212625332084, sortino=-0.5916854606052417, max_drawdown=-0.7957469820976257, mean=-0.0002609904622659087, vol=0.00871274434030056)

In [8]:
import pickle

OUT_PATH = Path("../signals/signal_files/wf_results.pkl")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with open(OUT_PATH, "wb") as f:
    pickle.dump(wf_results, f)

print("Saved wf_results to", OUT_PATH)

Saved wf_results to ../signals/signal_files/wf_results.pkl


In [ ]:
import pandas as pd

# Build a dummy price series from returns (start at 100)
price = 100 * (1 + pd.Series(returns)).cumprod()

pos_df = pd.DataFrame({
    "timestamp": pd.date_range("2000-01-01", periods=len(positions), freq="D"),
    "price": price.values,
    "target_position": positions,
})

pos_path = "../signals/signal_files/positions_example.csv"
pos_df.to_csv(pos_path, index=False)
print("Saved positions CSV to", pos_path)